# Proyecto 2 - Visión por Computadora
## Ejercicio 2
## Integrantes:

- Javier Alvarado - 21188
- Mario Guerra - 21008
- Emilio Solano - 21212

In [1]:
%pip install opencv-python mediapipe torch torchvision
!git clone https://github.com/ultralytics/yolov5
!cd yolov5
%pip install -r requirements.txt

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Usuario\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)


fatal: destination path 'yolov5' already exists and is not an empty directory.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
You should consider upgrading via the 'c:\Users\Usuario\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import os
import sys

yolo_path = "c:/Users/Usuario/OneDrive/Documentos/U/Año 5/Ciclo 1/Visión/Object_detectorVC/yolov5"
os.chdir(yolo_path)

if yolo_path not in sys.path:
    sys.path.append(yolo_path)

In [13]:
import os
import sys
import torch
import cv2
import mediapipe as mp
import numpy as np
from pathlib import Path

# 📁 Ruta donde está tu carpeta clonada de YOLOv5
yolo_path = "c:/Users/Usuario/OneDrive/Documentos/U/Año 5/Ciclo 1/Visión/Object_detectorVC/yolov5"
os.chdir(yolo_path)
if yolo_path not in sys.path:
    sys.path.append(yolo_path)

# ✅ Importaciones desde YOLOv5
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression, scale_boxes
from yolov5.utils.torch_utils import select_device
from yolov5.utils.augmentations import letterbox

# 🔧 Inicializar YOLOv5
device = select_device('')
model = DetectMultiBackend('./yolov5s.pt', device=device)
model.eval()

# 🕺 Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# 📹 Cargar el video
input_path = "../assets/danceVideo.mp4"
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 🎥 Inicializar salidas
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_pose = cv2.VideoWriter("../assets/silhouette_dance_video.mp4", fourcc, fps, (width, height))
out_black = cv2.VideoWriter("../assets/silhouette_dance_black.mp4", fourcc, fps, (width, height))

# 🔁 Procesar video frame por frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    original = frame.copy()
    black_frame = np.zeros_like(frame)

    # 📏 Preparar frame para YOLOv5
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_resized = letterbox(img, new_shape=640)[0]
    img_yolo = torch.from_numpy(img_resized).permute(2, 0, 1).float().to(device) / 255.0
    img_yolo = img_yolo.unsqueeze(0)

    # 🧠 Inferencia con YOLO
    pred = model(img_yolo)
    pred = non_max_suppression(pred, conf_thres=0.5, classes=[0])  # clase 0 = persona

    for det in pred:
        if det is not None and len(det):
            det[:, :4] = scale_boxes(img_resized.shape[:2], det[:, :4], frame.shape).round()

            for *xyxy, conf, cls in det:
                x1, y1, x2, y2 = map(int, xyxy)
                person_roi = frame[y1:y2, x1:x2]
                if person_roi.size == 0:
                    continue

                # 🧍 Estimar pose de la persona detectada
                person_rgb = cv2.cvtColor(person_roi, cv2.COLOR_BGR2RGB)
                result = pose.process(person_rgb)

                if result.pose_landmarks:
                    # Dibujar puntos y conexiones SOLO de esta persona, en un frame temporal
                    person_silhouette = np.zeros_like(frame)

                    # Convertir landmarks al espacio del frame original
                    pose_landmarks = result.pose_landmarks

                    # Ajustar coordenadas al contexto de toda la imagen (no solo ROI)
                    for idx, landmark in enumerate(pose_landmarks.landmark):
                        landmark.x = x1 / width + landmark.x * (x2 - x1) / width
                        landmark.y = y1 / height + landmark.y * (y2 - y1) / height

                    # Dibujar en el frame original
                    mp_drawing.draw_landmarks(
                        image=original,
                        landmark_list=pose_landmarks,
                        connections=mp_pose.POSE_CONNECTIONS,
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                    )

                    # Dibujar en el frame temporal (solo una persona)
                    mp_drawing.draw_landmarks(
                        image=person_silhouette,
                        landmark_list=pose_landmarks,
                        connections=mp_pose.POSE_CONNECTIONS,
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
                    )

                    # Sumar esta silueta individual al frame final
                    black_frame = cv2.add(black_frame, person_silhouette)


    out_pose.write(original)
    out_black.write(black_frame)

# 🧹 Liberar recursos
cap.release()
out_pose.release()
out_black.release()
cv2.destroyAllWindows()

YOLOv5  2025-4-9 Python-3.10.0 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
